# Music Recommendation System
## Part 2: Modeling & Recommendation Engine

This notebook implements **various embedding techniques and recommendation strategies** for our content-based music recommendation system. We compare TF-IDF, Word2Vec, and Sentence-BERT embeddings for lyric-based song similarity.

---

## Table of Contents

### Core Implementation
1. [Import Libraries](#1-import-libraries)
2. [Load Dataset](#2-load-dataset)
3. [Baseline: TF-IDF](#3-baseline-tf-idf)
4. [Building FAISS Index](#4-building-faiss-index-baseline)
5. [Recommendation Function](#5-recommendation-function)
6. [Model Persistence](#6-model-persistence)

### Improvements

#### Upgrade Song Embeddings
7. [Word2Vec / FastText](#7-improvement-1-word2vec--fasttext)
8. [Sentence-BERT](#8-improvement-2-sentence-bert)
9. [Audio Features (Spotify API)](#9-improvement-3-audio-features-spotify-api)
10. [Embedding Comparison](#10-embedding-comparison--evaluation)

#### Hybrid Recommendation
11. [Hybrid Recommendation System](#11-hybrid-recommendation-system)

#### Advanced FAISS (ANN)
12. [Advanced FAISS Indexes](#12-advanced-faiss-indexes)

---



## 1. Import Libraries

In [2]:
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from wordcloud import WordCloud
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import normalize
import faiss
import pickle
from scipy.sparse import save_npz
import gensim.downloader as api
from gensim.models import Word2Vec, FastText
import random
from tqdm import tqdm
from huggingface_hub import HfApi, login

## 2. Load Dataset

In [3]:
df = pd.read_csv('../data/processed/spotify_millsongdata_processed.csv')

## 3. Baseline: TF-IDF

> **Baseline Approach**: We start with TF-IDF (Term Frequency-Inverse Document Frequency) as our baseline embedding method. This is a classic approach for text similarity that works well but has limitations in capturing semantic meaning.

**TF-IDF (Term Frequency-Inverse Document Frequency)** is a numerical statistic that reflects how important a word is to a document in a collection.

- **Term Frequency (TF)**: How often a word appears in a document

$$
tf(t, d) = \frac{f_{t,d}}{\sum_{k} f_{k,d}}
$$

where $f_{t,d}$ denotes the number of occurrences of term $t$ in document $d$.

- **Inverse Document Frequency (IDF)**: How rare a word is across all documents

$$
idf(t, D) = \log \left( \frac{N}{1 + n_t} \right)
$$

where:
- $N$ is the total number of documents in the corpus,
- $n_t$ is the number of documents containing term $t$.


**TF-IDF Weight**

The final TF-IDF weight of term $t$ in document $d$ is computed as:

$$
w_{t,d} = tf(t, d) \times idf(t, D)
$$

Words that appear frequently in one song but rarely across all songs get higher weights, making them better for distinguishing between songs.

We limit to `max_features=5000` to keep the most important terms and reduce dimensionality.

In [4]:
tfidf_vectorizer = TfidfVectorizer(max_features=5000, dtype=np.float32)
tfidf_matrix = tfidf_vectorizer.fit_transform(df['cleaned_text'])

print(f"TF-IDF Matrix Shape: {tfidf_matrix.shape}")
print(f"Vocabulary Size: {len(tfidf_vectorizer.vocabulary_)}")

TF-IDF Matrix Shape: (57650, 5000)
Vocabulary Size: 5000


## 4. Building FAISS Index (Baseline)

**FAISS (Facebook AI Similarity Search)** is a library for efficient similarity search and clustering of dense vectors.

### **Why FAISS?**
- Traditional cosine similarity has O(n) complexity for each query
- FAISS uses optimized algorithms for faster nearest neighbor search
- Essential for scaling to large datasets (millions of songs)

---

### **Index Type: `IndexFlatIP`**
- **IP** = Inner Product (equivalent to cosine similarity for L2-normalized vectors)
- **Flat** = Exact search (no approximation)
- We normalize vectors with L2 norm so inner product equals cosine similarity

#### **$L_2$ Normalization**

$L_2$ normalization transforms a document vector  $\mathbf{x} = [x_1, x_2, \dots, x_n]$  into a unit vector $\hat{\mathbf{x}}$ with Euclidean norm equal to 1:

$$
\hat{\mathbf{x}} = \frac{\mathbf{x}}{\|\mathbf{x}\|_2}
= \frac{\mathbf{x}}{\sqrt{\sum_{i=1}^{n} x_i^2}}
$$

##### **Geometric Interpretation**

After $L_2$ normalization, all document vectors lie on the surface of a **unit hypersphere**. This normalization removes the influence of document length, ensuring that similarity comparisons depend only on the **direction** of the vectors (semantic content), rather than their magnitude.

---

#### **Similarity Search with FAISS and Inner Product**

In FAISS, `IndexFlatIP` performs similarity search based on the **Inner Product (IP)**.  
Given a query vector $\mathbf{q}$ and a set of document vectors $\mathbf{X}$ stored in the index, the similarity score $s_i$ is computed as:

$$
s_i = \mathbf{q} \cdot \mathbf{x}_i
= \sum_{j=1}^{dim} q_j \cdot x_{i,j}
$$

##### **Relationship Between Inner Product and Cosine Similarity**

Cosine Similarity between two vectors $\mathbf{q}$ and $\mathbf{x}$ is defined as:

$$
\text{cosine\_sim}(\mathbf{q}, \mathbf{x})
= \frac{\mathbf{q} \cdot \mathbf{x}}{\|\mathbf{q}\|_2 \, \|\mathbf{x}\|_2}
$$

Since both the query vector and document vectors are $L_2$-normalized  ($\|\mathbf{q}\|_2 = 1$ and $\|\mathbf{x}\|_2 = 1$), the equation simplifies to:

$$
\text{cosine\_sim}(\mathbf{q}, \mathbf{x})
= \mathbf{q} \cdot \mathbf{x}
= \text{Inner Product}
$$

Therefore, when using `IndexFlatIP` on $L_2$-normalized TF-IDF vectors, FAISS effectively computes **Cosine Similarity**, representing the degree of semantic similarity between the query text and the retrieved documents.

In [5]:
X = tfidf_matrix.toarray()
X = normalize(X, norm="l2")

dim = X.shape[1]
print(f"Vector Dimension: {dim}")

index = faiss.IndexFlatIP(dim)
index.add(X)

print(f"Total vectors in FAISS index: {index.ntotal}")
print("FAISS index built successfully!")

Vector Dimension: 5000
Total vectors in FAISS index: 57650
FAISS index built successfully!


## 5. Recommendation Function

The core recommendation logic:

1. **Find the song**: Look up the input song name in our dataset
2. **Get its vector**: Retrieve the TF-IDF vector for that song
3. **Search similar songs**: Use FAISS to find the k most similar songs
4. **Return results**: Return song details (excluding the input song itself)

In [6]:
def recommend_songs_tfidf(song_name, df=df, top_k=5):

    idx = df[df['song'].str.lower() == song_name.lower()].index
    if len(idx) == 0:
        return "Song not found in the dataset."
    idx = idx[0]

    query = X[idx].reshape(1, -1)

    scores, indices = index.search(query, top_k + 1)

    results = df[['song', 'artist']].iloc[indices[0][1:]].copy()
    results['similarity'] = scores[0][1:]

    return results

### Test the Recommendation System

Let's test our recommendation system by getting similar songs for the first song in our dataset.

In [7]:
idx = random.randint(0, len(df))
test_song = df['song'].iloc[idx]
test_artist = df['artist'].iloc[idx]
print(f"Getting recommendations for: '{test_song}' by {test_artist}")
print("-" * 50)

display(recommend_songs_tfidf(test_song))

Getting recommendations for: 'Girls It Ain't Easy' by Dusty Springfield
--------------------------------------------------


,song,artist,similarity
11631,It's So Easy (To Fall In Love),Linda Ronstadt,0.669506
40634,Easy As It Seems,Kiss,0.637069
11877,Easy,Lorde,0.632390
4904,It Ain't Gonna Be Easy,Elton John,0.605216
41729,It's So Easy,Linda Ronstadt,0.596836


## 6. Model Persistence

Save the trained models and processed data for later use:

- **df_cleaned.pkl**: Preprocessed DataFrame with cleaned lyrics
- **faiss_index.pkl**: FAISS index for fast similarity search
- **tfidf_matrix.pkl**: TF-IDF vectors for all songs

These files can be loaded later to make recommendations without retraining.

In [8]:
df.to_parquet("../data/processed/df_cleaned.parquet", index=False)
print("Saved: df_cleaned.parquet")

save_npz("../data/embeddings/embeddings_tfidf.npz", tfidf_matrix)
print("Saved: embeddings_tfidf.npz")

faiss.write_index(index, "../models/faiss_indexes/faiss_tfidf.index")
print("Saved: faiss_tfidf.index")

print("\nAll models saved successfully!")

Saved: df_cleaned.parquet
Saved: embeddings_tfidf.npz
Saved: faiss_tfidf.index

All models saved successfully!


---
# IMPROVEMENTS

The following sections implement advanced techniques to improve recommendation quality and performance.


## 7. Improvement 1: Word2Vec / FastText

> **Goal**: Capture semantic similarity between words. Unlike TF-IDF which treats words as independent tokens, word embeddings represent words in a continuous vector space where semantically similar words are close together.

### Why Word Embeddings?

**Limitation of TF-IDF vs Solution with Word2Vec/FastText:**
- "happy" and "joyful" are completely different features in TF-IDF, but similar vectors in Word2Vec
- TF-IDF has no semantic understanding, Word2Vec captures word meaning
- OOV words are ignored in TF-IDF, FastText handles via subwords

### Approaches

1. **Pre-trained Word2Vec** (Google News 300d)
   - 3M words, 300 dimensions
   - General semantic knowledge
   
2. **Custom FastText** (trained on lyrics)
   - Domain-specific vocabulary
   - Handles misspellings and slang


### 7.1 Tokenize Lyrics for Word Embeddings


In [9]:
df['tokens'] = df['cleaned_text'].apply(lambda x: x.split())

df[['song', 'text', 'tokens']]

,song,text,tokens
0,Ahe's My Kind Of Girl,"Look at her face, it's a wonderful face \r\nA...","[look, face, wonderful, face, means, something..."
1,"Andante, Andante","Take it easy with me, please \r\nTouch me gen...","[take, easy, please, touch, gently, like, summ..."
2,As Good As New,I'll never know why I had to go \r\nWhy I had...,"[ill, never, know, go, put, lousy, rotten, sho..."
3,Bang,Making somebody happy is a question of give an...,"[making, somebody, happy, question, give, take..."
4,Bang-A-Boomerang,Making somebody happy is a question of give an...,"[making, somebody, happy, question, give, take..."
...,...,...,...
57645,Good Old Days,Irie days come on play \r\nLet the angels fly...,"[irie, days, come, play, let, angels, fly, let..."
57646,Hand To Mouth,Power to the workers \r\nMore power \r\nPowe...,"[power, workers, power, power, workers, need, ..."
57647,Come With Me,all you need \r\nis something i'll believe \...,"[need, something, ill, believe, flashlights, h..."
57648,Desire,northern star \r\nam i frightened \r\nwhere ...,"[northern, star, frightened, go, rest, cant, s..."


### 7.2 Option A: Using Pre-trained Word2Vec (Google News)

Using pre-trained Word2Vec model trained on Google News (3 million words, 300-dimensional vectors).

**Pros:** 
- High-quality embeddings trained on massive corpus
- Captures general semantic relationships

**Cons:**
- Large download (~1.5GB)
- May miss domain-specific music vocabulary

In [10]:
try:
    w2v_model = api.load('word2vec-google-news-300')
    print(f"Vocabulary size: {len(w2v_model.key_to_index):,}")
    print(f"Vector dimension: {w2v_model.vector_size}")
except Exception as e:
    print(f"Error loading model: {e}")

Vocabulary size: 3,000,000
Vector dimension: 300


### 7.3 Document Embedding Function

To get a single vector representation for an entire song (document), we compute the **weighted average** of word vectors:

$$\vec{d} = \frac{1}{|W|} \sum_{w \in W} \vec{w}$$

where $W$ is the set of words in the document and $\vec{w}$ is the word vector.

In [11]:
def get_document_embedding(tokens, model, vector_size=300):
    """
    Compute document embedding by averaging word vectors.
    
    Args:
        tokens: List of words in the document
        model: Word2Vec/FastText model
        vector_size: Dimension of word vectors
    
    Returns:
        numpy array of shape (vector_size,) - document embedding
    """

    word_vectors = []
    for word in tokens:
        try:
            word_vectors.append(model[word])
        except KeyError:
            continue
     
    if len(word_vectors) == 0:
        return np.zero(vector_size)
    
    return np.mean(word_vectors, axis=0)

In [12]:
sample_tokens = df['tokens'].iloc[0]
sample_embedding = get_document_embedding(sample_tokens, w2v_model)
print(f"Sample document embedding shape: {sample_embedding.shape}")
print(f"First 10 values: {sample_embedding[:10]}")

Sample document embedding shape: (300,)
First 10 values: [ 0.03464193  0.01799033  0.04964281  0.09320665 -0.0741931  -0.0054287
  0.05288998 -0.10220951  0.0597229   0.04583878]


### 7.4 Build Document Embeddings for All Songs

In [13]:
doc_embedding_w2v = []
for tokens in tqdm(df['tokens'], desc="Processing songs"):
    embedding = get_document_embedding(tokens, w2v_model)
    doc_embedding_w2v.append(embedding)

X_w2v = np.array(doc_embedding_w2v, dtype=np.float32)

print(f"Document embeddings shape: {X_w2v.shape}")
print(f"Memory usage: {X_w2v.nbytes / 1024 / 1024:.2f} MB")

Processing songs: 100%|██████████| 57650/57650 [00:08<00:00, 7089.69it/s]

Document embeddings shape: (57650, 300)
Memory usage: 65.98 MB


### 7.5 Option B: Train Custom FastText Model


FastText extends Word2Vec by representing words as bags of character n-grams. This allows it to:
- Handle **out-of-vocabulary (OOV)** words by composing subword vectors
- Better handle **misspellings** and **slang** common in lyrics
- Capture **morphological** relationships (e.g., "love", "loving", "loved")

**Training Parameters:**
- `vector_size=100`: Dimension of word vectors (smaller for faster training)
- `window=5`: Context window size
- `min_count=2`: Ignore words that appear less than 2 times
- `epochs=10`: Number of training iterations
- `sg=1`: Use Skip-gram (better for small datasets)

In [14]:
fasttext_model = FastText(
    sentences=df['tokens'].to_list(),
    vector_size=100,
    window=5,
    min_count=2,
    workers=4,
    sg=1,
    epochs=10
)

print(f"FastText model trained successfully!")
print(f"Vocabulary size: {len(fasttext_model.wv.key_to_index):,}")
print(f"Vector dimension: {fasttext_model.wv.vector_size}")

Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_fl

FastText model trained successfully!
Vocabulary size: 52,953
Vector dimension: 100


### 7.6 Explore Semantic Relationships

Let's verify that our FastText model captured meaningful semantic relationships.

In [15]:
test_words = ['love', 'heart', 'dance', 'night', 'baby']

print("Words most similar to common lyrics terms:\n")
for word in test_words:
    try:
        similar = fasttext_model.wv.most_similar(word, topn=5)
        similar_words = [f"{w} ({s:.3f})" for w, s in similar]
        print(f"'{word}' → {', '.join(similar_words)}")
    except KeyError:
        print(f"'{word}' not in vocabulary")

Words most similar to common lyrics terms:

'love' → mylove (0.785), alove (0.782), ohlove (0.779), itlove (0.769), lovelove (0.758)
'heart' → heartfelt (0.803), heartmy (0.787), heartheart (0.763), hearts (0.742), hearttheres (0.726)
'dance' → dancego (0.800), decadance (0.759), dancefloor (0.742), danceevery (0.725), dancins (0.714)
'night' → nightgo (0.801), nightynight (0.791), nightoh (0.782), nightspot (0.769), nightnight (0.763)
'baby' → ohbaby (0.813), ofbaby (0.803), bbaby (0.798), saidbaby (0.788), babyi (0.783)


### 7.7 Build FastText Document Embeddings

Now let's create document embeddings using our trained FastText model.

In [16]:
print("Computing FastText document embeddings for all songs...")

doc_embeddings_ft = []
for tokens in tqdm(df['tokens'], desc="Processing songs"):
    embedding = get_document_embedding(tokens, fasttext_model.wv, vector_size=100)
    doc_embeddings_ft.append(embedding)

X_ft = np.array(doc_embeddings_ft, dtype=np.float32)

print(f"\nFastText document embeddings shape: {X_ft.shape}")
print(f"Memory usage: {X_ft.nbytes / 1024 / 1024:.2f} MB")

Computing FastText document embeddings for all songs...


Processing songs: 100%|██████████| 57650/57650 [00:09<00:00, 5975.92it/s]


FastText document embeddings shape: (57650, 100)
Memory usage: 21.99 MB


### 7.8 Build FAISS Indexes for Semantic Search

Create FAISS indexes for both Word2Vec and FastText embeddings.

In [17]:
# Normalize embeddings for cosine similarity
X_w2v_norm = normalize(X_w2v, norm='l2')
X_ft_norm = normalize(X_ft, norm='l2')

# Build FAISS index for Word2Vec embeddings
print("Building FAISS index for Word2Vec embeddings...")
index_w2v = faiss.IndexFlatIP(X_w2v_norm.shape[1])
index_w2v.add(X_w2v_norm)
print(f"Word2Vec FAISS index: {index_w2v.ntotal} vectors, {X_w2v_norm.shape[1]} dimensions")

# Build FAISS index for FastText embeddings
print("\nBuilding FAISS index for FastText embeddings...")
index_ft = faiss.IndexFlatIP(X_ft_norm.shape[1])
index_ft.add(X_ft_norm)
print(f"FastText FAISS index: {index_ft.ntotal} vectors, {X_ft_norm.shape[1]} dimensions")

Building FAISS index for Word2Vec embeddings...
Word2Vec FAISS index: 57650 vectors, 300 dimensions

Building FAISS index for FastText embeddings...
FastText FAISS index: 57650 vectors, 100 dimensions


### 7.9 Semantic Recommendation Functions

Create recommendation functions using Word2Vec and FastText embeddings.

In [18]:
def recommend_songs_w2v(song_name, df=df, top_k=5):
    """
    Recommend songs using Word2Vec embeddings.
    """
    idx = df[df['song'].str.lower() == song_name.lower()].index
    if len(idx) == 0:
        return "Song not found in the dataset."
    idx = idx[0]
    
    query = X_w2v_norm[idx].reshape(1, -1)
    scores, indices = index_w2v.search(query, top_k + 1)
    
    result = df[['song', 'artist']].iloc[indices[0][1:]].copy()
    result['similarity'] = scores[0][1:]
    return result


def recommend_songs_fasttext(song_name, df=df, top_k=5):
    """
    Recommend songs using FastText embeddings.
    """
    idx = df[df['song'].str.lower() == song_name.lower()].index
    if len(idx) == 0:
        return "Song not found in the dataset."
    idx = idx[0]
    
    query = X_ft_norm[idx].reshape(1, -1)
    scores, indices = index_ft.search(query, top_k + 1)
    
    result = df[['song', 'artist']].iloc[indices[0][1:]].copy()
    result['similarity'] = scores[0][1:]
    return result


### 7.10 Compare All Methods: TF-IDF vs Word2Vec vs FastText

In [19]:
idx = random.randint(0, len(df))
test_song = df['song'].iloc[idx]
test_artist = df['song'].iloc[idx]

print(f"Recommendations for: '{test_song}' by {test_artist}")
print("=" * 70)

print("\nTF-IDF Recommendations:")
print("-" * 40)
display(recommend_songs_tfidf(test_song))

print("\nWord2Vec Recommendations:")
print("-" * 40)
display(recommend_songs_w2v(test_song))

print("\nFastText Recommendations:")
print("-" * 40)
display(recommend_songs_fasttext(test_song))

Recommendations for: 'A Moment Suspended In Time' by A Moment Suspended In Time

TF-IDF Recommendations:
----------------------------------------


,song,artist,similarity
40633,Dreamin',Kiss,0.484948
49061,Dreamin' About U,Prince,0.446586
44759,I Must Be Dreaming,Nat King Cole,0.445657
13767,I Must Be Dreaming,Neil Sedaka,0.404857
49275,A Winter's Tale,Queen,0.400612



Word2Vec Recommendations:
----------------------------------------


,song,artist,similarity
23900,Right Before Your Eyes,America,0.949501
25783,Can't Wait,Bob Dylan,0.949051
22643,Leaving Me,Zox,0.948582
52601,Not The Moment,Supertramp,0.948522
55012,Finding Me,Vertical Horizon,0.948123



FastText Recommendations:
----------------------------------------


,song,artist,similarity
34925,Coma,Guns N' Roses,0.975415
14789,Out Of Control,Oingo Boingo,0.974774
22464,Wasted,Zebrahead,0.974675
22436,Morse Code For Suckers,Zebrahead,0.973370
33411,Behind The Lines,Genesis,0.973218


### 7.11 Save Semantic Models

In [20]:
# Save FastText model
fasttext_model.save("../models/fasttext/fasttext_lyrics.model")
print("Saved: fasttext_lyrics.model")

# Save FastText embeddings
np.save("../data/embeddings/embeddings_fasttext.npy", X_ft)
print("Saved: embeddings_fasttext.npy")

# Save Word2Vec embeddings (from pre-trained model)
np.save("../data/embeddings/embeddings_w2v.npy", X_w2v)
print("Saved: embeddings_w2v.npy")

# Save FAISS indexes
faiss.write_index(index_w2v, "../models/faiss_indexes/faiss_w2v.index")
print("Saved: faiss_w2v.index")

faiss.write_index(index_ft, "../models/faiss_indexes/faiss_fasttext.index")
print("Saved: faiss_fasttext.index")

print("\nAll semantic models saved successfully!")

Saved: fasttext_lyrics.model
Saved: embeddings_fasttext.npy
Saved: embeddings_w2v.npy
Saved: faiss_w2v.index
Saved: faiss_fasttext.index

All semantic models saved successfully!


---

## 8. Improvement 2: Sentence-BERT

> **Goal**: Use transformer-based embeddings that understand entire sentences/paragraphs, not just individual words.

### Why Sentence-BERT?

**Word2Vec Limitation vs SBERT Solution:**
- Word2Vec averages word vectors (loses word order), SBERT encodes full sentence context
- "I love you" = "You love I" in Word2Vec, but SBERT preserves word order meaning
- Word2Vec has fixed vocabulary, SBERT handles any text

### Model Options

1. **`all-MiniLM-L6-v2`** (recommended)
   - 384 dimensions
   - Fast inference
   - Good quality

2. **`all-mpnet-base-v2`** (higher quality)
   - 768 dimensions
   - Slower but more accurate


### 8.1 Install and Load Sentence-Transformers


In [21]:
# TODO: Install sentence-transformers
# !pip install sentence-transformers -q
# from sentence_transformers import SentenceTransformer
# sbert_model = SentenceTransformer('all-MiniLM-L6-v2')


### 8.2 Generate SBERT Embeddings for All Songs


In [22]:
# TODO: Encode all lyrics with SBERT
# X_sbert = sbert_model.encode(df['cleaned_text'].tolist(), show_progress_bar=True)


### 8.3 Build FAISS Index & Recommendation Function


In [23]:
# TODO: Build FAISS index for SBERT embeddings
# TODO: Implement recommend_songs_sbert()


---

## 9. Improvement 3: Audio Features (Spotify API)

> **Goal**: Incorporate audio characteristics (tempo, energy, danceability) to capture musical similarity beyond lyrics.

### Spotify Audio Features

- `danceability`: How suitable for dancing (0.0 - 1.0)
- `energy`: Intensity and activity (0.0 - 1.0)
- `tempo`: Beats per minute (~50 - 200)
- `valence`: Musical positiveness (0.0 - 1.0)
- `acousticness`: Acoustic vs electronic (0.0 - 1.0)
- `instrumentalness`: Vocal vs instrumental (0.0 - 1.0)

### Implementation Steps

1. Set up Spotify API credentials
2. Fetch audio features for each song
3. Normalize and combine with lyrics embeddings
4. Create hybrid similarity metric


### 9.1 Set Up Spotify API


In [24]:
# TODO: Install spotipy and set up credentials
# !pip install spotipy -q
# import spotipy
# from spotipy.oauth2 import SpotifyClientCredentials
# sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id="...", client_secret="..."))


### 9.2 Fetch Audio Features for Songs


In [25]:
# TODO: Search for tracks and get audio features
# def get_audio_features(artist, song):
#     results = sp.search(q=f"artist:{artist} track:{song}", type='track', limit=1)
#     if results['tracks']['items']:
#         track_id = results['tracks']['items'][0]['id']
#         return sp.audio_features(track_id)[0]
#     return None


### 9.3 Create Audio Feature Embeddings


In [26]:
# TODO: Normalize and combine audio features into embeddings
# audio_features = ['danceability', 'energy', 'tempo', 'valence', 'acousticness', 'instrumentalness']
# X_audio = df[audio_features].values
# X_audio = normalize(X_audio, norm='l2')


---

## 10. Embedding Comparison & Evaluation

> **Goal**: Compare TF-IDF, Word2Vec, and SBERT embeddings for lyric-based song similarity. Evaluate which method produces the most meaningful recommendations.

### Evaluation Metrics

- **Qualitative**: Manual inspection of recommendation quality
- **Diversity**: How diverse are the recommended songs?
- **Artist Coverage**: Do recommendations include different artists?
- **Semantic Coherence**: Do lyrics share similar themes?


### 10.1 Side-by-Side Comparison


In [27]:
# TODO: Compare recommendations from all methods for the same song
# def compare_recommendations(song_name, top_k=5):
#     print(f"Recommendations for: {song_name}")
#     print("\n📊 TF-IDF:")
#     display(recommend_songs_tfidf(song_name, top_k))
#     print("\n📊 Word2Vec:")
#     display(recommend_songs_w2v(song_name, top_k))
#     print("\n📊 Sentence-BERT:")
#     display(recommend_songs_sbert(song_name, top_k))


### 10.2 Embedding Visualization (t-SNE/UMAP)


In [28]:
# TODO: Visualize embeddings in 2D using t-SNE or UMAP
# from sklearn.manifold import TSNE
# X_embedded = TSNE(n_components=2).fit_transform(X_sbert[:1000])
# plt.scatter(X_embedded[:, 0], X_embedded[:, 1], alpha=0.5)
# plt.title("SBERT Embeddings (t-SNE)")


### 10.3 Summary Table


In [29]:
# TODO: Create summary comparison table
# comparison_df = pd.DataFrame({
#     'Method': ['TF-IDF', 'Word2Vec', 'FastText', 'SBERT'],
#     'Dimensions': [5000, 300, 100, 384],
#     'Semantic': ['❌', '✅', '✅', '✅✅'],
#     'Speed': ['Fast', 'Fast', 'Fast', 'Slow'],
#     'Quality': ['Good', 'Better', 'Better', 'Best']
# })


---

## 11. Hybrid Recommendation System

> **Goal**: Combine multiple similarity signals (lyrics, genre, artist, popularity) for better recommendations.

### Hybrid Score Formula

$$\text{FinalScore} = w_1 \cdot \text{LyricSim} + w_2 \cdot \text{GenreSim} + w_3 \cdot \text{Popularity}$$

**Default Weights:**
- w1 = 0.5 (Lyrics similarity - most important)
- w2 = 0.3 (Genre/Artist similarity)
- w3 = 0.2 (Popularity score)

### Components to Combine

- **Lyrics Similarity**: SBERT/TF-IDF cosine similarity (0.5)
- **Artist Similarity**: Same/similar artist bonus (0.1)
- **Genre Similarity**: Same genre bonus (0.2)
- **Popularity**: Normalized play count (0.1)
- **Release Year**: Recency bonus (0.1)


### 11.1 Lyrics Similarity Component


In [30]:
# TODO: Get lyrics similarity scores from SBERT index
# def get_lyrics_similarity(song_idx, candidate_indices):
#     query = X_sbert[song_idx].reshape(1, -1)
#     scores, _ = index_sbert.search(query, len(candidate_indices))
#     return scores[0]


### 11.2 Artist / Genre Similarity Component


In [31]:
# TODO: Compute artist/genre similarity
# def get_artist_similarity(song_idx, candidate_indices):
#     query_artist = df.iloc[song_idx]['artist']
#     similarities = []
#     for idx in candidate_indices:
#         if df.iloc[idx]['artist'] == query_artist:
#             similarities.append(1.0)  # Same artist
#         else:
#             similarities.append(0.0)
#     return np.array(similarities)


### 11.3 Popularity / Release Year Component


In [32]:
# TODO: Compute popularity scores (if available)
# def get_popularity_scores(candidate_indices):
#     if 'popularity' in df.columns:
#         scores = df.iloc[candidate_indices]['popularity'].values
#         return (scores - scores.min()) / (scores.max() - scores.min())
#     return np.zeros(len(candidate_indices))


### 11.4 Hybrid Recommendation Function


In [33]:
# TODO: Implement hybrid recommendation
# def recommend_songs_hybrid(song_name, top_k=5, weights={'lyrics': 0.5, 'genre': 0.3, 'popularity': 0.2}):
#     idx = df[df['song'].str.lower() == song_name.lower()].index[0]
#     
#     # Get initial candidates from lyrics similarity
#     _, candidates = index_sbert.search(X_sbert[idx].reshape(1, -1), 100)
#     candidates = candidates[0][1:]  # Exclude query song
#     
#     # Compute component scores
#     lyrics_scores = get_lyrics_similarity(idx, candidates)
#     artist_scores = get_artist_similarity(idx, candidates)
#     popularity_scores = get_popularity_scores(candidates)
#     
#     # Combine with weights
#     final_scores = (weights['lyrics'] * lyrics_scores + 
#                     weights['genre'] * artist_scores + 
#                     weights['popularity'] * popularity_scores)
#     
#     # Return top-k
#     top_indices = np.argsort(final_scores)[::-1][:top_k]
#     return df.iloc[candidates[top_indices]][['song', 'artist']]


### 11.5 Weight Tuning & Optimization


In [34]:
# TODO: Test different weight combinations
# weight_configs = [
#     {'lyrics': 0.5, 'genre': 0.3, 'popularity': 0.2},
#     {'lyrics': 0.7, 'genre': 0.2, 'popularity': 0.1},
#     {'lyrics': 0.4, 'genre': 0.4, 'popularity': 0.2},
# ]
# 
# for config in weight_configs:
#     print(f"Weights: {config}")
#     display(recommend_songs_hybrid(test_song, weights=config))


---

## 12. Advanced FAISS Indexes

> **Goal**: Compare different FAISS index types for speed vs accuracy trade-offs.

### Index Types Comparison

- **IndexFlatIP**: Exact search (baseline) - Slow, High Memory, 100% Recall
- **IndexIVFFlat**: Inverted file index - Fast, Medium Memory, ~95% Recall
- **IndexHNSWFlat**: Hierarchical NSW graph - Very Fast, Medium Memory, ~98% Recall
- **IndexIVFPQ**: Product quantization - Very Fast, Low Memory, ~90% Recall

### When to Use Each

- **IndexFlatIP**: Small datasets (<100K), need exact results
- **IndexIVFFlat**: Medium datasets, good balance
- **IndexHNSWFlat**: Large datasets, best quality/speed
- **IndexIVFPQ**: Very large datasets, memory constrained


### 12.1 Baseline: IndexFlatIP (Current)


In [35]:
# Baseline: Exact search
# index_flat = faiss.IndexFlatIP(dim)
# index_flat.add(X)
# print(f"IndexFlatIP: {index_flat.ntotal} vectors")


### 12.2 IndexIVFFlat (Inverted File Index)


In [36]:
# TODO: Build IndexIVFFlat
# nlist = 100  # Number of clusters
# quantizer = faiss.IndexFlatIP(dim)
# index_ivf = faiss.IndexIVFFlat(quantizer, dim, nlist, faiss.METRIC_INNER_PRODUCT)
# index_ivf.train(X)
# index_ivf.add(X)
# index_ivf.nprobe = 10  # Number of clusters to search
# print(f"IndexIVFFlat: {index_ivf.ntotal} vectors, {nlist} clusters")


### 12.3 IndexHNSWFlat (Hierarchical NSW)


In [37]:
# TODO: Build IndexHNSWFlat
# M = 32  # Number of connections per layer
# index_hnsw = faiss.IndexHNSWFlat(dim, M, faiss.METRIC_INNER_PRODUCT)
# index_hnsw.hnsw.efConstruction = 40  # Construction-time parameter
# index_hnsw.hnsw.efSearch = 16  # Search-time parameter
# index_hnsw.add(X)
# print(f"IndexHNSWFlat: {index_hnsw.ntotal} vectors, M={M}")


### 12.4 Speed Comparison Benchmark


In [38]:
# TODO: Benchmark search speed
# import time
# 
# def benchmark_index(index, name, X, n_queries=100, top_k=10):
#     queries = X[np.random.choice(len(X), n_queries)]
#     start = time.time()
#     for q in queries:
#         index.search(q.reshape(1, -1), top_k)
#     elapsed = time.time() - start
#     qps = n_queries / elapsed
#     print(f"{name}: {qps:.1f} queries/sec ({elapsed*1000/n_queries:.2f} ms/query)")
#     return qps
# 
# benchmark_index(index_flat, "IndexFlatIP", X)
# benchmark_index(index_ivf, "IndexIVFFlat", X)
# benchmark_index(index_hnsw, "IndexHNSWFlat", X)


### 12.5 Memory Usage Comparison


In [39]:
# TODO: Compare memory usage
# import sys
# 
# def get_index_size(index):
#     faiss.write_index(index, "/tmp/temp_index")
#     size_mb = os.path.getsize("/tmp/temp_index") / 1024 / 1024
#     return size_mb
# 
# print(f"IndexFlatIP: {get_index_size(index_flat):.2f} MB")
# print(f"IndexIVFFlat: {get_index_size(index_ivf):.2f} MB")
# print(f"IndexHNSWFlat: {get_index_size(index_hnsw):.2f} MB")


### 12.6 Recall@K Evaluation


In [40]:
# TODO: Compute Recall@K (how many of exact top-K are found by ANN)
# def compute_recall(index_exact, index_approx, X, n_queries=100, k=10):
#     queries = X[np.random.choice(len(X), n_queries)]
#     recalls = []
#     for q in queries:
#         q = q.reshape(1, -1)
#         _, exact_ids = index_exact.search(q, k)
#         _, approx_ids = index_approx.search(q, k)
#         recall = len(set(exact_ids[0]) & set(approx_ids[0])) / k
#         recalls.append(recall)
#     return np.mean(recalls)
# 
# print(f"IndexIVFFlat Recall@10: {compute_recall(index_flat, index_ivf, X):.2%}")
# print(f"IndexHNSWFlat Recall@10: {compute_recall(index_flat, index_hnsw, X):.2%}")


### 12.7 Performance Comparison Chart


In [41]:
# TODO: Create visualization comparing all indexes
# results = pd.DataFrame({
#     'Index': ['IndexFlatIP', 'IndexIVFFlat', 'IndexHNSWFlat'],
#     'Speed (QPS)': [100, 1000, 5000],
#     'Memory (MB)': [500, 550, 600],
#     'Recall@10': [1.0, 0.95, 0.98]
# })
# 
# fig, axes = plt.subplots(1, 3, figsize=(15, 4))
# results.plot.bar(x='Index', y='Speed (QPS)', ax=axes[0], color='steelblue')
# results.plot.bar(x='Index', y='Memory (MB)', ax=axes[1], color='coral')
# results.plot.bar(x='Index', y='Recall@10', ax=axes[2], color='seagreen')
# plt.tight_layout()
# plt.show()


---

## 13. Upload Models to Hugging Face Hub

Upload all embeddings, FAISS indexes, and models to Hugging Face for easy deployment.

In [42]:
login()  # This will prompt for your token

# Initialize API
api = HfApi()

# Your Hugging Face username/repo
REPO_ID = "namviet157/music-recommendation"  # Change this to your repo

# Create repository if it doesn't exist
try:
    api.create_repo(repo_id=REPO_ID, repo_type="model", exist_ok=True)
    print(f"Repository '{REPO_ID}' ready!")
except Exception as e:
    print(f"Repo exists or error: {e}")


Repository 'namviet157/music-recommendation' ready!


In [43]:
# Upload all files to Hugging Face Hub

files_to_upload = [
    # DataFrame
    ("../data/processed/df_cleaned.parquet", "df_cleaned.parquet"),
    
    # Embeddings
    ("../data/embeddings/embeddings_tfidf.npz", "embeddings_tfidf.npz"),
    ("../data/embeddings/embeddings_fasttext.npy", "embeddings_fasttext.npy"),
    ("../data/embeddings/embeddings_w2v.npy", "embeddings_w2v.npy"),
    
    # FAISS indexes
    ("../models/faiss_indexes/faiss_tfidf.index", "faiss_tfidf.index"),
    ("../models/faiss_indexes/faiss_fasttext.index", "faiss_fasttext.index"),
    ("../models/faiss_indexes/faiss_w2v.index", "faiss_w2v.index"),

    # requirements.txt
    ("../requirements.txt", "requirements.txt"),
]

print("Uploading files to Hugging Face Hub...")
for local_path, hf_path in files_to_upload:
    try:
        api.upload_file(
            path_or_fileobj=local_path,
            path_in_repo=hf_path,
            repo_id=REPO_ID,
            repo_type="model"
        )
        print(f"Uploaded: {hf_path}")
    except Exception as e:
        print(f"Failed to upload {hf_path}: {e}")

print("\nAll files uploaded successfully!")


Uploading files to Hugging Face Hub...


d:\miniconda3\envs\min_ds-env\Lib\site-packages\huggingface_hub\hf_api.py:4180: UserWarning: It seems that you are about to commit a data file (df_cleaned.parquet) to a model repository. You are sure this is intended? If you are trying to upload a dataset, please set `repo_type='dataset'` or `--repo-type=dataset` in a CLI.
  warnings.warn(


Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

No files have been modified since last commit. Skipping to prevent empty commit.


...

Uploaded: faiss_fasttext.index


Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

No files have been modified since last commit. Skipping to prevent empty commit.


Uploaded: faiss_w2v.index
Uploaded: requirements.txt

All files uploaded successfully!


---

## Final Summary

### Implementation Progress

**Core (Complete):**
- TF-IDF Baseline
- FAISS IndexFlatIP
- Basic Recommendation

**Embedding (TODO):**
- Word2Vec/FastText
- Sentence-BERT
- Audio Features
- Comparison & Eval

**Hybrid (TODO):**
- Multi-signal Fusion

**FAISS (TODO):**
- IndexIVFFlat
- IndexHNSWFlat
- Benchmarks

### Key Files

```
models/
├── faiss_indexes/
│   ├── faiss.index          # TF-IDF baseline
│   ├── faiss_sbert.index    # Sentence-BERT (TODO)
│   └── faiss_hnsw.index     # HNSW optimized (TODO)
├── fasttext_lyrics.model    # Custom FastText (TODO)
└── sbert_embeddings.npy     # SBERT embeddings (TODO)
```

### Next Steps

1. Implement Word2Vec/FastText embeddings
2. Add Sentence-BERT for best quality
3. Build hybrid recommendation system
4. Optimize with advanced FAISS indexes
5. Deploy as web application
